In [1]:
import pandas as pd
import pyomo.environ as pyo
import requests as req
from dotenv import load_dotenv
from sklearn.preprocessing import minmax_scale
import os

In [2]:
# Melakukan loading data client dan driver dari CSV

clients = pd.read_csv('../datasets/data_client_small.csv')
drivers = pd.read_csv('../datasets/data_driver_small.csv')
banned = pd.read_csv('../datasets/banned_data_small.csv')

drivers = drivers.drop(['total_trip', 'time_idle', 'order_count'], axis=1)

In [3]:
# Simulasi tidak balanced antara driver dan client.
drivers = drivers[0:4]

In [4]:
# Max latitude, longitude dari dummy harus bisa menggambarkan situasi terburuk (posisi terjauh)
# Tapi juga menggambarkan posisi driver kurang lebih. Jadi dummy itu merepresentasikan jarak terjauh dari rata-rata driver.
driver_lat_avg = drivers["latitude"].mean()
driver_long_avg = drivers["longitude"].mean()

client_lat_avg = clients["latitude"].mean()
client_long_avg = clients["longitude"].mean()

# Cek apakah data balanced/imbalanced
if len(clients) > len(drivers):
	# Add dummy drivers
	addition = abs(len(clients)) - abs(len(drivers))
	for i in range(addition):
		print(drivers.tail(1).iloc[0]['id'] + 1)
		drivers = drivers.append({
			'id': drivers.tail(1).iloc[0]['id'] + 1,
			'latitude': driver_lat_avg,
			'longitude': driver_long_avg,
			'rating_driver': 0.0,
		}, ignore_index=True)

elif len(clients) < len(drivers):
	# Add dummy clients
	addition = abs(len(clients)) - abs(len(drivers))
	for i in range(addition):
		clients = clients.append({
			'id': clients.tail(1).iloc[0]['id'] + 1,
			'latitude': client_lat_avg,
			'longitude': client_long_avg,
			'rating_client': 0.0
		}, ignore_index=True)

drivers

14.0
15.0


C:\Users\matth\AppData\Local\Temp\ipykernel_768\3085633750.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  drivers = drivers.append({
C:\Users\matth\AppData\Local\Temp\ipykernel_768\3085633750.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  drivers = drivers.append({


,id,latitude,longitude,rating_driver
0,10.0,-7.301530,112.781522,3.0
1,11.0,-7.257839,112.790380,4.0
2,12.0,-7.364964,112.774871,2.0
3,13.0,-7.274133,112.652447,5.0
4,14.0,-7.299616,112.749805,0.0
5,15.0,-7.299616,112.749805,0.0


In [5]:
# Normalisasi
clients['rating_client_scaled'] = minmax_scale(clients['rating_client'])
drivers['rating_driver_scaled'] = minmax_scale(drivers['rating_driver'])

In [6]:
drivers_data = drivers.to_dict('records')
clients_data = clients.to_dict('records')
banned_data = banned.to_dict('records')

# No banning
# banned_data = None

clients_data, drivers_data, banned_data

([{'id': 20,
   'latitude': -7.250317999983876,
   'longitude': 112.68898399990732,
   'rating_client': 5,
   'rating_client_scaled': 1.0},
  {'id': 21,
   'latitude': -7.349470999997735,
   'longitude': 112.69903299988196,
   'rating_client': 1,
   'rating_client_scaled': 0.0},
  {'id': 22,
   'latitude': -7.312413999992556,
   'longitude': 112.76892399970548,
   'rating_client': 4,
   'rating_client_scaled': 0.75},
  {'id': 23,
   'latitude': -7.332430999995354,
   'longitude': 112.67340899994664,
   'rating_client': 4,
   'rating_client_scaled': 0.75},
  {'id': 24,
   'latitude': -7.268699999986445,
   'longitude': 112.65469599999388,
   'rating_client': 2,
   'rating_client_scaled': 0.25},
  {'id': 25,
   'latitude': -7.283033999988449,
   'longitude': 112.7348569997915,
   'rating_client': 5,
   'rating_client_scaled': 1.0}],
 [{'id': 10.0,
   'latitude': -7.301529999991034,
   'longitude': 112.78152199967369,
   'rating_driver': 3.0,
   'rating_driver_scaled': 0.6000000000000001}

In [7]:
# Load env untuk store API key
load_dotenv(override=True)

# Memo untuk tidak memanggil API untuk data yang sama.
memo = {}

In [8]:
from sklearn.preprocessing import minmax_scale

# Function untuk request ke ORS
def create_matrix(data, locations):
    durations_matrix, distances_matrix = {}, {}
    durations_matrix_ori, distances_matrix_ori = {}, {}

    # SCALING
    data['durations_original'] = data['durations']
    data['distances_original'] = data['distances']

    data['durations_scaled'] = minmax_scale(data['durations'])
    data['distances_scaled'] = minmax_scale(data['distances'])

    for i, src in enumerate(locations):
        distances_matrix[src] = {}
        durations_matrix[src] = {}
        distances_matrix_ori[src] = {}
        durations_matrix_ori[src] = {}

        for j, dest in enumerate(locations):
            durations_matrix[src][dest] = data['durations_scaled'][i][j]
            distances_matrix[src][dest] = data['distances_scaled'][i][j]
            durations_matrix_ori[src][dest] = data['durations_original'][i][j]
            distances_matrix_ori[src][dest] = data['distances_original'][i][j]

    return distances_matrix, durations_matrix, distances_matrix_ori, durations_matrix_ori

def get_location_info(locations):
    data = memo.get(tuple(locations))

    if data != None:
        print("Cache hit")
        result = create_matrix(data, locations)
        memo[tuple(locations)] = data
        return result

    key = os.getenv('OPENROUTESERVICE_KEY')
    headers = {
        'Authorization': key,
        'Content-Type': 'application/json; charset=utf-8',
        'Accept': 'application/json, application/geo+json',
    }
    body = {
        'locations': [[i for i in locs] for locs in locations],
        'metrics': ['distance', 'duration'],
    }

    res = req.post(
        'https://api.openrouteservice.org/v2/matrix/driving-car',
        json=body,
        headers=headers
    )
    
    print(res.status_code)
    if res.status_code == 200:
        data = res.json()
        memo[tuple(locations)] = data
        return create_matrix(data, locations)
    else:
        raise Exception("Error", res.status_code, res.content.decode())

locations = []
for i in drivers_data:
    locations.append((i['longitude'], i['latitude']))
for i in clients_data:
    locations.append((i['longitude'], i['latitude']))

distances, durations, distances_ori, durations_ori = get_location_info(locations)

200


In [9]:
# Definisikan weight.
# Weight pada kondisi positif dan negatif, negatif berarti cenderung untuk lebih dipilih karena
#  menggunakan metode Minimize pada rumus.

weights = {
    'distance'      : { 'target': 0, 'positive': 1, 'negative': -1 },
    'duration'      : { 'target': 0, 'positive': 1, 'negative': -1 },
    'rating'        : { 'target': 0, 'positive': 1, 'negative': -1 },
}

In [10]:
# Karena data yang berpengaruh hanya data yang digabungkan antara keduanya, 
#   maka kami mengumpulkan data setelah dirumuskan dari masing-masing pasangan penumpang-driver
data: dict[tuple, dict[str, any]] = {}
for d in drivers_data:
    d_id = d['id']
    data[d_id] = {}
    for c in clients_data:
        c_id = c['id']

        distance = distances[(d['longitude'], d['latitude'])][(c['longitude'], c['latitude'])]
        duration = durations[(d['longitude'], d['latitude'])][(c['longitude'], c['latitude'])]

        distance_ori = distances_ori[(d['longitude'], d['latitude'])][(c['longitude'], c['latitude'])]
        duration_ori = durations_ori[(d['longitude'], d['latitude'])][(c['longitude'], c['latitude'])]

        rating = abs(d['rating_driver_scaled'] - c['rating_client_scaled'])

        data[d_id][c_id] = {
            "distance": distance,
            "duration": duration,
            
            "rating": rating,

            # For validation
            "rating_client": c['rating_client_scaled'],
            "rating_driver": d['rating_driver_scaled'],

            "distance_ori": distance_ori,
            "duration_ori": duration_ori,

            "rating_client_ori": c['rating_client'],
            "rating_driver_ori": d['rating_driver'],
        }

data

{10.0: {20: {'distance': 0.7091946623755295,
   'duration': 0.7846033909607069,
   'rating': 0.3999999999999999,
   'rating_client': 1.0,
   'rating_driver': 0.6000000000000001,
   'distance_ori': 22617.7,
   'duration_ori': 1600.23,
   'rating_client_ori': 5,
   'rating_driver_ori': 3.0},
  21: {'distance': 0.643368979665982,
   'duration': 0.6462874612705987,
   'rating': 0.6000000000000001,
   'rating_client': 0.0,
   'rating_driver': 0.6000000000000001,
   'distance_ori': 14591.75,
   'duration_ori': 942.83,
   'rating_client_ori': 1,
   'rating_driver_ori': 3.0},
  22: {'distance': 0.1602256949021469,
   'duration': 0.23824505575994068,
   'rating': 0.1499999999999999,
   'rating_client': 0.75,
   'rating_driver': 0.6000000000000001,
   'distance_ori': 2917.21,
   'duration_ori': 330.92,
   'rating_client_ori': 4,
   'rating_driver_ori': 3.0},
  23: {'distance': 0.6746161433794187,
   'duration': 0.7363057975828501,
   'rating': 0.1499999999999999,
   'rating_client': 0.75,
   'ra

In [11]:
# Model pyomo utk solve GP
model = pyo.ConcreteModel()
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# Variabel yang digunakan untuk mengidentifikasi driver dan customer adalah ID.
drivers = [d['id'] for d in drivers_data]
clients = [c['id'] for c in clients_data]

# Mendaftarkan semua kombinasi antara ID driver dan client.
model.x = pyo.Var(drivers, clients, domain=pyo.NonNegativeIntegers)

# Rumus untuk menghitung penalty negatif dan positif
def get_penalty(target, val, positive=True):
  if positive:
    if val > target: return val - target
    else: return 0
  else:
    if val < target: return target - val
    else: return 0

# Penalty = (weight positive * X * deviasi dgn target apabila >) + (weight negatif * X * deviasi dgn target apabila <)
# Salah satu dari deviasi ini akan menjadi 0 (target: 3, val: 5 maka sisi negatif akan 0)
def penalty(c, d, param):
  m = weights[param]
  return (
    (m['positive'] * model.x[d, c] * get_penalty(m['target'], data[d][c][param], True)) + 
    (m['negative'] * model.x[d, c] * get_penalty(m['target'], data[d][c][param], False))
  )

# Hitung penalti dari matching yang sesuai (DV jadi 1.0)
# Ini dipakai di paling terakhir untuk menampilkan perhitungan penalti.
def v_penalty(c, d, param):
  m = weights[param]
  return (
    (m['positive'] * 1 * get_penalty(m['target'], data[d][c][param], True)) + 
    (m['negative'] * 1 * get_penalty(m['target'], data[d][c][param], False))
  )

# Objective function: minimalkan jumlah dari semua penalty masing-masing kolom data gabungan.
model.Cost = pyo.Objective(
    expr = sum([
        penalty(c, d, 'distance') + 
        penalty(c, d, 'duration') +
        penalty(c, d, 'rating')
        for c in clients for d in drivers
      ]),
    sense = pyo.minimize)

# Constraint yang digunakan yaitu pada jumlah nilai variabel pada setiap driver dengan semua kombinasi pasangan penumpangnya antara 0 atau 1 karena hanya memperbolehkan 1 driver mengambil 1 customer atau tidak mengambil customer sama sekali.

model.driver = pyo.ConstraintList()
for d in drivers:
  model.driver.add(sum(model.x[d, c] for c in clients) == 1)

model.client = pyo.ConstraintList()
for c in clients:
  model.client.add(sum(model.x[d, c] for d in drivers) == 1)

# Menambahkan constraint banned untuk pasangan penumpang dan driver yang tidak diperbolehkan ada di hasil matching.
if banned_data is not None:
  model.banned = pyo.ConstraintList()
  for i in banned_data:
    if i['client_id'] in clients and i['driver_id'] in drivers:
      model.banned.add(model.x[i['driver_id'], i['client_id']] == 0)

In [12]:
# Gunakan solver utk solve
result = pyo.SolverFactory('cbc').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 7.48856681
  Upper bound: 7.48856681
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 35
  Number of nonzeros: 35
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of

In [13]:
# Menampilkan hasil matching dalam bentuk table (dari bawaan DataFrame)

print("Hasil dari Assignment Matching\n")
df = pd.DataFrame([], columns=clients, index=drivers)

for c in clients:
    mdr = []
    for d in drivers:
        mdr.append(model.x[d, c]())
    df[c] = mdr

df


Hasil dari Assignment Matching



,20,21,22,23,24,25
10.0,0.0,0.0,1.0,0.0,0.0,0.0
11.0,0.0,0.0,0.0,0.0,0.0,1.0
12.0,0.0,0.0,0.0,1.0,0.0,0.0
13.0,1.0,0.0,0.0,0.0,0.0,0.0
14.0,0.0,0.0,0.0,0.0,1.0,0.0
15.0,0.0,1.0,0.0,0.0,0.0,0.0


In [14]:
# Menampilkan hasil matching dan list penalti.
from pprint import pprint

result_dict = df.to_dict("dict")

for i in result_dict:
    for j in result_dict[i]:
        s = result_dict[i][j]
        if s == 1.0:
            print(f"{j} ==> {i}")
            print(f"Distance: {data[j][i]['distance_ori']}")
            print(f"Duration: {data[j][i]['duration_ori']}")
print()

def find_pen(i):
    for j in clients:
        print(f"---- Penalties for Driver {i} -> Client {j}: ")
        distance_pen = v_penalty(j, i, "distance")
        duration_pen = v_penalty(j, i, "duration")
        rating_pen = v_penalty(j, i, "rating")
        print("\tDistance:", data[i][j]["distance_ori"])
        print("\tDistance Penalty:", distance_pen)
        print()
        print("\tDuration:", data[i][j]["duration_ori"])
        print("\tDuration Penalty:", duration_pen)
        print()
        print("\tRating:", data[i][j]["rating"])
        print("\tRating Driver:", data[i][j]["rating_driver_ori"])
        print("\tRating Driver (Scaled):", data[i][j]["rating_driver"])

        print("\tRating Client:", data[i][j]["rating_client_ori"])
        print("\tRating Client (Scaled):", data[i][j]["rating_client"])
        print("\tRating Penalty:", rating_pen)
        print()
        print("\tTotal penalty:", distance_pen + duration_pen + rating_pen)
        print()

for i in drivers:
    print(f"Driver {i}:")
    find_pen(i)
    for j in clients:
        if result_dict[j][i] == 1:
            print(f"Match: Driver {i} -> Client {j}")
    print()

"""
Kenapa penalty 3.0:
Karena distance terjauh -> duration terlama (2.0)
Rating 0 dengan 5
"""

13.0 ==> 20
Distance: 8382.64
Duration: 989.06
15.0 ==> 21
Distance: 11414.95
Duration: 761.84
10.0 ==> 22
Distance: 2917.21
Duration: 330.92
12.0 ==> 23
Distance: 22393.24
Duration: 1582.89
14.0 ==> 24
Distance: 16779.25
Duration: 1213.01
11.0 ==> 25
Distance: 11969.56
Duration: 904.96

Driver 10.0:
---- Penalties for Driver 10.0 -> Client 20: 
	Distance: 22617.7
	Distance Penalty: 0.7091946623755295

	Duration: 1600.23
	Duration Penalty: 0.7846033909607069

	Rating: 0.3999999999999999
	Rating Driver: 3.0
	Rating Driver (Scaled): 0.6000000000000001
	Rating Client: 5
	Rating Client (Scaled): 1.0
	Rating Penalty: 0.3999999999999999

	Total penalty: 1.8937980533362362

---- Penalties for Driver 10.0 -> Client 21: 
	Distance: 14591.75
	Distance Penalty: 0.643368979665982

	Duration: 942.83
	Duration Penalty: 0.6462874612705987

	Rating: 0.6000000000000001
	Rating Driver: 3.0
	Rating Driver (Scaled): 0.6000000000000001
	Rating Client: 1
	Rating Client (Scaled): 0.0
	Rating Penalty: 0.60000

'\nKenapa penalty 3.0:\nKarena distance terjauh -> duration terlama (2.0)\nRating 0 dengan 5\n'

In [16]:
# Validation
import itertools as it

combinations = []
permut = it.permutations(drivers, len(clients))
for i in permut:
	zipped = zip(i, clients)
	combinations.append(list(zipped))

if banned_data is not None:
	for i in banned_data:
		combinations = list(filter(lambda x: (i['driver_id'], i['client_id']) not in x, combinations))

all_comb_penalty = []
for comblist in combinations:
	cp = []
	for pair in comblist:
		dist_p = v_penalty(pair[1], pair[0], "distance")
		dur_p = v_penalty(pair[1], pair[0], "duration")
		rat_p = v_penalty(pair[1], pair[0], "rating")
		total_penalty = dist_p + dur_p + rat_p

		print(f"-- Driver {pair[0]} <--> Client {pair[1]} --")
		print(f"Distance Penalty: {dist_p}")
		print(f"Duration Penalty: {dur_p}")
		print(f"Rating Penalty: {rat_p}")
		print(f"Total penalty: {total_penalty}")
		print()

		cp.append(total_penalty)

	all_comb_penalty.append({"combinations": comblist, "penalty": sum(cp)})

minimum = {"penalty": 999}
for i in all_comb_penalty:
	if i['penalty'] < minimum['penalty']:
		minimum = i

print(minimum)

-- Driver 10.0 <--> Client 20 --
Distance Penalty: 0.7091946623755295
Duration Penalty: 0.7846033909607069
Rating Penalty: 0.3999999999999999
Total penalty: 1.8937980533362362

-- Driver 11.0 <--> Client 21 --
Distance Penalty: 1.0
Duration Penalty: 1.0
Rating Penalty: 0.8
Total penalty: 2.8

-- Driver 12.0 <--> Client 22 --
Distance Penalty: 0.393452365259726
Duration Penalty: 0.44921129741754806
Rating Penalty: 0.35
Total penalty: 1.192663662677274

-- Driver 13.0 <--> Client 23 --
Distance Penalty: 0.5401806403027728
Duration Penalty: 0.6193934128829497
Rating Penalty: 0.25
Total penalty: 1.4095740531857226

-- Driver 14.0 <--> Client 24 --
Distance Penalty: 0.5740336354956119
Duration Penalty: 0.6616538482517864
Rating Penalty: 0.25
Total penalty: 1.4856874837473983

-- Driver 15.0 <--> Client 25 --
Distance Penalty: 0.3233973644885206
Duration Penalty: 0.3272479447970354
Rating Penalty: 1.0
Total penalty: 1.650645309285556

-- Driver 10.0 <--> Client 20 --
Distance Penalty: 0.7091

In [32]:
df
data

{10.0: {20: {'distance': 0.7091946623755295,
   'duration': 0.7846033909607069,
   'rating': 0.3999999999999999,
   'rating_client': 1.0,
   'rating_driver': 0.6000000000000001,
   'distance_ori': 22617.7,
   'duration_ori': 1600.23,
   'rating_client_ori': 5,
   'rating_driver_ori': 3.0},
  21: {'distance': 0.643368979665982,
   'duration': 0.6462874612705987,
   'rating': 0.6000000000000001,
   'rating_client': 0.0,
   'rating_driver': 0.6000000000000001,
   'distance_ori': 14591.75,
   'duration_ori': 942.83,
   'rating_client_ori': 1,
   'rating_driver_ori': 3.0},
  22: {'distance': 0.1602256949021469,
   'duration': 0.23824505575994068,
   'rating': 0.1499999999999999,
   'rating_client': 0.75,
   'rating_driver': 0.6000000000000001,
   'distance_ori': 2917.21,
   'duration_ori': 330.92,
   'rating_client_ori': 4,
   'rating_driver_ori': 3.0},
  23: {'distance': 0.6746161433794187,
   'duration': 0.7363057975828501,
   'rating': 0.1499999999999999,
   'rating_client': 0.75,
   'ra